In [88]:
# WHAT'S NEW? 
#
# Running Simualted VQE Locally
# Importation and Handling of Data
# Builds three part Hamiltonian
# 1) revenue based incentive (return)
# 2) convolution matrix based decentive (volatility + interconnectedness => risk)
# 3) penalty term based budget decentive
# Generalized cost_funct(params) with interchangable ANSATZ and Estimator() type
#

In [17]:
############# LIBRARY IMPORTS #############
###########################################
###########################################
###########################################

In [11]:
# General imports
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import copy

# Circuit needs
from qiskit import *
from qiskit.visualization import *
from qiskit.circuit.library import *
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session
from qiskit.primitives import Sampler, Estimator, StatevectorEstimator, StatevectorSampler
from qiskit.circuit import Parameter, QuantumCircuit
import time

# Dynamic Decoupling
from qiskit.transpiler import InstructionProperties
from qiskit.circuit.library import XGate, YGate

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.scheduling import (
    ALAPScheduleAnalysis,
    PadDynamicalDecoupling,
)

from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary as sel
from qiskit.transpiler.passes import BasisTranslator

# Pauli Strings
from qiskit.quantum_info import Pauli,SparsePauliOp

In [13]:
# Loading IBM Quantum account
service = QiskitRuntimeService(channel="ibm_quantum", token="07b1ac63155b68dbe096e7b2d8b7483a25fdd267ae57bc80d736778c14d559fc7d10d9b8cd2780bf8df1e3a384b13aff2dc7cc2a4d159e3443efd700542b37de")
backend = service.least_busy(operational=True, simulator=False)
print(backend)

<IBMBackend('ibm_brisbane')>


In [23]:
############### FUNCTIONS #################
###########################################
###########################################
###########################################

In [90]:
######################################################### GENERATING BASE DATA ######################################################################

#####################################################################################################################################################
def base_data(holder):
    global p, P, r, u, convolu, d, conver, size
    
    # PRICE
    p = []
    for i in holder:
        p.append(i)
        
    # CURRENT PRICE
    P = []
    for i in holder:
        P.append(i[-1])
    
    # REVENUE
    r = []
    for i in holder:
        r.append(reven(i))

    # AVERAGE REVENUE
    u = []
    for i in r:
        u.append(avg(i))

    # CONVOLUTION MATRIX
    convolu = np.zeros((len(holder), len(holder)))

    for i in range(len(holder)):
        for j in range(len(holder)):
            convolu[i,j] = inner(add(r[i],-u[i]), add(r[j],-u[j]))

    # CONVERT TO BITS
    d = []
    for i in range(len(holder)):
        d.append(round(np.log2(int(B/P[i])+1)))
        
    size = sum(d)

    # REVERSION MATRIX
    conver = np.zeros((len(d), sum(d)))
    
    k = 0
    for i in range(len(d)):
        for j in range(d[i]):
            conver[i,k] = 2**j
            k += 1

In [35]:
######################################################### GENERATING HAMILTONIAN ####################################################################

#####################################################################################################################################################
def rev_the_hamiltonian():
    global h, h_, hi, J, J_, J__, ji, pi, pi_, bi
    I = []
    for i in range(sum(d)):
        I.append("I")
    
    ############### hi Zi ################
    h = (np.array([u])*np.array([P])/B)
    h_ = []
    for i in range(len(d)):
        for j in range(d[i]):
            h_.append(h[0][i]*(2**j))
        
    hi = []
    for i in range(sum(d)):
        I[i] = "Z"
        hi.append(stitch(I))
        I[i] = "I"
    ######################################
    
    ############# ji Zi Zj ###############
    J = ((convolu*(np.array([P])*(1/B)).reshape(-1, 1)).T)*(np.array([P])/B).reshape(-1, 1)
    J_ = np.dot(conver.T,(np.dot(J, conver)))
    
    J__ = []
    ji = []
    for i in range(sum(d)):
        for j in range(sum(d)):
            I[i] = "Z"
            I[j] = "Z"
            if j != i:
                ji.append(stitch(I))
                J__.append(J_[i,j])
            I[i] = "I"
            I[j] = "I"
    ######################################
    
    ########### Budget Penalty ###########
    pi = (np.array([P])*1/B)
    pi_ = []
    for i in range(len(d)):
        for j in range(d[i]):
            pi_.append(pi[0][i]*(2**j))

    bi = copy.copy(hi)
    ######################################

In [37]:
def norm(b):
    v = copy.copy(b)
    for i in range(len(v)):
        v[i] /= sum(v)
    return v, sum(v)

def avg(x):
    avg = sum(x)/len(x)
    return avg

def reven(v):
    b = []
    for i in range(1,len(v)):
        b.append(v[i]-v[i-1])
    return b

def add(b, r):
    v = copy.copy(b)
    for i in range(len(v)):
        v[i] += r
    return v

def multip(b, r):
    v = copy.copy(b)
    for i in range(len(v)):
        v[i] *= r
    return v

def addvec(b,c):
    v = copy.copy(b)
    w = copy.copy(c)
    for i in range(len(v)):
        w[i] += v[i]
    return w

def inner(a, b):
    total = 0
    for i in range(len(a)):
        total += a[i] * b[i]
    return total

def stitch(v):
    s = ""
    for i in v:
        s += i
    return s

def dict_to_vec(dict):
    keys_list = list(dict.keys())

    vector = []
    for i in range(2**sum(d)):
        vector.append(dict.get(i, 0))
        
    return vector

In [39]:
def fetch_investment_vector(params):
    with Session(backend=backend) as session:
        ansatz = EfficientSU2(size)
        ansatz.measure_all()
        
        pub = (ansatz, params)
        sampler = StatevectorSampler()
        
        shots = (sum(d))*1000
        job = sampler.run([pub], shots=shots)
        result = job.result()[0]
        
        dict = result.data.meas.get_int_counts()
        max_key = max(dict, key=dict.get)
        
        binary = bin(max_key)[2:]
        binary_ = "0"*(sum(d)-len(binary))+binary
    
        tempvec = np.zeros((1,sum(d)))
        for i in range(sum(d)):
            tempvec[0,i] += int(binary_[i])
        
        xvec = np.dot(conver, tempvec[0])
        return xvec

In [94]:
################# IMPORT ##################
###########################################
###########################################
###########################################

In [82]:
############################################################# VARIABLES #############################################################################

#####################################################################################################################################################

B = 400 # Budget
R = 1 # Risk Ratio
delta = 2 # Penalty Term


In [80]:
######################################################### IMPORT DATA HERE ##########################################################################

#####################################################################################################################################################

import pandas as pd
df = pd.read_csv('sample.csv')

holder = []
for i in range(1,5):
    holder.append(df.iloc[:, i].tolist())

holder2 = []
for i in holder:
    holder2.append(i[0:int(len(holder[0])/24)]) # scale the amount of data

base_data(holder2)
rev_the_hamiltonian()


In [70]:
############################################################ VQE OPTIMIZATION #######################################################################

################################################# COBYLA, QUANTUM SIMULATION (CLASSICAL)  ###########################################################


def cost_funct(params):
    ############################# ANSATZ LIVES HERE ##################################
    ansatz = EfficientSU2(size)    
    ##################################################################################
    
    observables = [
    [SparsePauliOp(hi, h_)], [SparsePauliOp(ji, J__)], [SparsePauliOp(bi, pi_)]
    ]
    
    estimator = StatevectorEstimator() # Simulating a Quantum Computer
    pub = (ansatz, observables, params)
    job = estimator.run([pub])
    result = job.result()
    
    energy_cost = result[0].data.evs[0] + R * result[0].data.evs[1] + delta * (result[0].data.evs[2] - (2-sum(pi_)*(1/B)))**2
    E.append(energy_cost[0])
    parames.append(params)
    
    return energy_cost[0]

In [92]:


with Session(backend=backend) as session:
    E = []
    parames = []

    x0 = 2 * np.pi * np.random.random(size*8)
    
    start_time = time.time()
    result = minimize(cost_funct, x0, method='COBYLA', options={'maxiter':5})
    end_time = time.time()
    
    opt_params = parames[E.index(max(E))]
    opt_energy = E[E.index(max(E))]
    
    print("time for operation", end_time-start_time)
    opt_x = fetch_investment_vector(opt_params)
    print("optimal vector:", opt_x)
    print("money invested:", np.dot(P, opt_x))

plt.plot(E, marker='o', linestyle='-', color='b')
plt.grid(True)
plt.xlabel('Iteration')
plt.ylabel('Energy')
plt.title('COBYLA')

plt.show()


KeyboardInterrupt: 